# Pregunta 1

Se requiere elaborar un sistema de predicción de temperatura para las distintas zonas de en una ciudad. En cada zona existe una plataforma capaz de predecir el tiempo hasta 10 días en el futuro. Esta plataforma requiere de la siguiente información para hacer la predicción: la temperatura actual en su zona local, medida a través de un sensor, y la temperatura actual en las zonas vecinas. Existe un servidor que envía a cada zona una lista de tuplas serializadas mediante \texttt{pickle}, donde cada tupla incluye el host y el puerto de cada equipo de medici\'on en las zonas vecinas a cada nodo, de tal forma que puedan conectarse e intercambiar la información necesaria. Cada vez que la temperatura en alguna zona cambia, debe realizarse una nueva predicción para el d\'ia en cuesti\'on y los diez d\'ias subsiguientes de cada zona afectada. Cada zona no puede acceder en forma directa al valor de los sensores de sus vecinos, pero si al valor del sensor que está dentro de la misma zona.

Suponga que existe una función que recibe como argumentos la información de temperatura de su zona y la de los vecinos, y retorna la temperatura para los próximos 10 días en la zona.

Implemente la solución correspondiente que trabajar\'a en cada una de las estaciones de medida de la ciudad. Asuma que el servidor que retorna la lista con las zonas vecinas ya existe y usted sólo debe preocuparse de conectarse a él. También suponga que la función que realiza la predicción está implementada y que existe una función que retorna el valor del sensor para cada zona.

In [1]:
import socket
import pickle
import threading
import sys


MAX_SIZE = 1024


class Estacion:
    
    def __init__(self):
        self.lista_vecinos = list()       
        self.conectar_servidor_central()
        
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.socket.bind(self.getlocalhost())
        self.socket.listen()
        
        self.t = threading.Thread(target=self.recibir_conexiones, daemon=True)
        self.start()
    
    def conectar_servidor_central(self):
        socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        try:
            socket.connect(('servidor_central.ing.puc.cl', 80))
            # Asumimos que la lista tiene un tamaño maximo de 1024 bytes
            data = socket.recv(MAX_SIZE)
            self.lista_vecinos = pickle.loads(data)
        except socket.error:
            print('ERROR')
            self.conexion = False
            sys.exit()
        finally:
            socket.close()
    
    def recibir_conexiones(self):
        self.conexiones = list()
        while True:
            socket_vecino, address = self.socket.accept()
            self.conexiones.append(
                threading.Thread(target=self.escuchar, 
                                 args=(socket_vecino, address), daemon=True))
            self.conexiones[-1].start()
            
    def escuchar(self, socket_vecino, address):
        while True:
            try:
                data = self.socket.recv(MAX_SIZE)
                # Asumimos que existe este metodo
                self.calcular_nueva_temperatura()
                self.enviar_temperatura_nueva(address)
            except socket.timeout:
                break
                
    def enviar_temperatura_nueva(self, address):
        for host, port in self.lista_vecinos:
            if (host, port) != address:
                # Se asume que existe
                data = pickle.dumps(self.temperatura)
                self.socket.connect((host, port))
                self.socket.send(data)

## Establecer conexión con Servidor Central

|Punto| **Requerimientos**                                | **Puntaje** |
|---|------------------------------------------------|:-----------:|
| A| Configuración socket |  **2 pto** |
| B| Usar connect| **1 pto**  |
| C| Recibir la información de los vecinos|**1 pto**|
| D| Deserializar bytes | **2 pto**|
| E| Manejo de errores | **1 pto**|
||**Total**| **7**|

## Establecer conexión con cada uno de sus vecinos

|Punto| **Requerimientos**             | **Puntaje** |
|---|---------------------------------|:-----------:|
| A| Configuración socket servidor |  **1 pto** |
| B| Hacer bind| **1 pto**  |
| C| Hacer accept|**1 pto**|
| D| Uso de threads| **2 pto**|
| E| Guardar sockets| **1 pto**|
||**Total**| **6**|

## Propagar temperatura cuando cambie

|Punto| **Requerimientos**                                | **Puntaje** |
|---|------------------------------------------------|:-----------:|
| A| Calcular nueva temperatura con el método que ya existe|  **1 pto** |
| B| Propagar a los vecinos| **2 pto**  |
||**Total**| **3**|

## Recibir temperatura de los vecinos

|Punto| **Requerimientos**                                | **Puntaje** |
|---|------------------------------------------------|:-----------:|
| A| Escuchar cada uno de los vecinos | **1 pto**|
| B| Recibir la información de los vecinos|**1 pto**|
| C| Deserializar bytes | **1 pto**|
| D| Manejo de errores| **1 pto**|
| E| Uso de Threads| **2 ptos**
||**Total**| **6**|

# Pregunta 2

Considerando el mismo contexto de la pregunta anterior, suponga que usted debe implementar el servidor que se encarga de enviar a cada nodo la lista con la información necesaria de sus nodos vecinos. El servidor debe recibir como input un nodo del grafo que representa las conexiones entre las zonas (ejemplo, ver la figura). Considere que el nodo contiene: un host, un puerto y una lista con los nodos vecinos en el grafo.

%Asuma que el grafo corresponde a un conjunto de nodos $\{ v_1, v_2, \ldots, v_N \}$ y aristas $\{ \epsilon_{ij} \}$ ($i,j \in {1,2, \ldots, N}$) , donde dos nodos $\{ v_i, v_j \}$ están directamente conectados si es que existe la %arista $\epsilon_{ij}$ en el grafo.

![Grafo](./grafo.png?raw=true)

In [3]:
from random import choice, randint

class Nodo:
    
    def __init__(self, identificador):
        self.host = "192.169.0.{}".format(identificador)
        self.port = 5350
        self.identificador = identificador
        self.vecinos = list()
    
    def __repr__(self):
        return str(self.identificador)
    
    
Grafos = dict()

for i in range(1, 10):
    Grafos[i] = Nodo(i)
    
Grafos[1].vecinos = [Grafos[7], Grafos[6], Grafos[2]]
Grafos[2].vecinos = [Grafos[1], Grafos[6], Grafos[3]]
Grafos[3].vecinos = [Grafos[2], Grafos[4], Grafos[5]]
Grafos[4].vecinos = [Grafos[3], Grafos[5], Grafos[8], Grafos[9]]
Grafos[5].vecinos = [Grafos[3], Grafos[4]]
Grafos[6].vecinos = [Grafos[7], Grafos[1], Grafos[2]]
Grafos[7].vecinos = [Grafos[1], Grafos[6], Grafos[8]]
Grafos[8].vecinos = [Grafos[7], Grafos[4], Grafos[9]]
Grafos[9].vecinos = [Grafos[8], Grafos[4]]

In [5]:
import pickle
import socket
from collections import deque


class ServidorCentral:
    def __init__(self, nodo):
        # Construir grafo
        self.vecinos = dict()
        self.construir_grafo(nodo)
        
        print(self.vecinos)

        # Inicializacion del socket
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.host = socket.gethostname()
        self.port = 10001
        self.socket.bind((self.host, self.port))
        self.socket.listen(20)
        self.escuchar()

    def escuchar(self):
        while True:
            socket_cliente, address = self.socket.accept()
            self.enviar_vecinos(address, socket_cliente)
            # Aca se podria crar un thread para responder
            # A solicitudes de los nodos.
            # Esto no es necesario para la pregunta 2

    def construir_grafo(self, nodo):
        por_visitar = deque()
        por_visitar.append(nodo)
        visitados = set()

        while por_visitar:
            actual = por_visitar.popleft()

            if actual not in visitados:
                # Recorrer nodo actual
                visitados.add(actual)
                address = (actual.host, actual.port)
                self.vecinos[address] = actual.vecinos

                # Agregar vecinos
                for vecino in actual.vecinos:
                    por_visitar.append(vecino)

    def enviar_vecinos(self, address, socket):
        vecinos = self.vecinos[address]
        socket.send(pickle.dumps(vecinos))


In [6]:
nodo = choice(Grafos)
sc = ServidorCentral(nodo)

## Networking

|Punto| **Requerimientos**                                | **Puntaje** |
|---|------------------------------------------------|:-----------:|
| A| Creación y configuración socket |  **2 pto** |
| B| Codificar mensaje| **1 pto**  |
| C| Envío correcto del mensaje|**1 pto**|
||**Total**| **7**|

## EDD

|Punto| **Requerimientos**             | **Puntaje** |
|---|---------------------------------|:-----------:|
| A| Recorrer todos los nodos |  **1 pto** |
| B| A| **1 pto**  |
| C| Hacer accept|**1 pto**|
| D| Uso de threads| **2 pto**|
| E| Guardar sockets| **1 pto**|
||**Total**| **6**|

## Propagar temperatura cuando cambie

|Punto| **Requerimientos**                                | **Puntaje** |
|---|------------------------------------------------|:-----------:|
| A| Calcular nueva temperatura con el método que ya existe|  **1 pto** |
| B| Propagar a los vecinos| **2 pto**  |
||**Total**| **3**|

## Recibir temperatura de los vecinos

|Punto| **Requerimientos**                                | **Puntaje** |
|---|------------------------------------------------|:-----------:|
| A| Escuchar cada uno de los vecinos | **1 pto**|
| B| Recibir la información de los vecinos|**1 pto**|
| C| Deserializar bytes | **1 pto**|
| D| Manejo de errores| **1 pto**|
| E| Uso de Threads| **2 ptos**
||**Total**| **6**|